#Prepare before Experiments

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

Found GPU at: /device:GPU:0


<tf.Tensor: shape=(), dtype=float32, numpy=1542.8479>

In [4]:
!pip install transformers>=3.0.2
!pip install tqdm
!pip install torch>=1.2.0
!pip install scipy
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.3-cp36-none-any.whl size=97299 sha256=6eff280dd42887a5bd02d2ba0bc8ec1e639c2e24ec2109390308ae2142a609a9
  Stored in directory: /root/.cache/pip/wheels/75/d6/0a/cab163b21d0597cc1580bc344487b11ad405e0d1d314725f2b
Successfully built sentence-transformers


## Dataset Preparing

PLEASE PREPARE THE ARGUMENTS (SENTENCES) LIST.

What kind of list???


[
  (
    KP, [Arguments which are included in KP]
  )
]

where KP is the centre sentence (if you don't have the centre sentence/summary, please place all of documents here), and Arguments refer sentence in usual summarization tasks.

In [4]:
root_path ="/content/drive/My Drive/ArgumentKeyPoint/"

In [5]:
import csv
import numpy as np
import random
from sentence_transformers import SentenceTransformer

In [6]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [7]:
def MakeSameKPArguments(filepath):
    """
    This function will return 
            [ (KP, [arguments which are labeled to the same KP]
               )
            ]
    """ 
    with open(filepath, encoding='utf-8') as f:
        reader = csv.reader(f)
        result_data = [row for row in reader]
    new_data = []
    for d in result_data:
        if d[4] == "1":#d[4]=label
            new_data.append(d)
    del result_data

    KPList = []
    results = {} #dictionary
    for d in new_data:
      Argument = d[1]
      KP = d[2]
      if KP not  in KPList:
          KPList.append(KP)
          results[KP] = [Argument]
      else:
          ArgumentsListOnSameKP = results[KP]
          ArgumentsListOnSameKP.append(Argument)
          results[KP] = ArgumentsListOnSameKP
    del KPList
    del new_data
    #return results
    new = list(results.items())
    return new

In [9]:
train2path = root_path + "data/train.csv"
test2path = root_path + "data/test.csv"
dev2path = root_path + "data/dev.csv"
OriginalTrainData = MakeSameKPArguments(train2path)
OriginalTestData = MakeSameKPArguments(test2path)
OriginalDevnData = MakeSameKPArguments(dev2path)

In [10]:
print(len(OriginalTrainData))
print(len(OriginalTestData))
print(len(OriginalDevnData))

142
61
40


In [11]:
numArguments = []
for eachtuple in OriginalTrainData:
  arguments = eachtuple[1]
  numArguments.append(len(arguments))
for eachtuple in OriginalTestData:
  arguments = eachtuple[1]
  numArguments.append(len(arguments))
for eachtuple in OriginalDevnData:
  arguments = eachtuple[1]
  numArguments.append(len(arguments))

print("Argumentsの数の最大値：　" + str(max(numArguments)))
print("Argumentsの数の最小値：　" + str(min(numArguments)))
print("Argumentsの数の平均値：　" + str(sum(numArguments)/len(numArguments)))

Argumentsの数の最大値：　95
Argumentsの数の最小値：　3
Argumentsの数の平均値：　20.567901234567902


## Dataset Details

Train Dataset: 142

Test Dataset: 61

Dev Dataset: 40



*   平均: 20.57
*   中央値: 13.00
*   分散: 376.66
*   標準偏差: 19.41
*   最頻値：[(4, 17), (3, 15), (6, 13), (7, 12), (5, 12)]

In [12]:
from statistics import mean, median,variance,stdev
data = numArguments
m = mean(data)
median = median(data)
variance = variance(data)
stdev = stdev(data)
print('平均: {0:.2f}'.format(m))
print('中央値: {0:.2f}'.format(median))
print('分散: {0:.2f}'.format(variance))
print('標準偏差: {0:.2f}'.format(stdev))
from collections import Counter
c = Counter(data)
print("最頻値：")
print(c.most_common(5))

平均: 20.57
中央値: 13.00
分散: 376.66
標準偏差: 19.41
最頻値：
[(4, 17), (3, 15), (6, 13), (7, 12), (5, 12)]


#Unsupervised Methods

In [13]:
from sentence_transformers import SentenceTransformer
model = 'bert-large-nli-mean-tokens'
embedder = SentenceTransformer(model)
print("=== Model " + model + " loaded ===")

100%|██████████| 1.24G/1.24G [00:23<00:00, 51.9MB/s]


=== Model bert-large-nli-mean-tokens loaded ===


In [14]:
train2path = root_path + "data/train.csv"
test2path = root_path + "data/test.csv"
dev2path = root_path + "data/dev.csv"
OriginalTrainData = MakeSameKPArguments(train2path)
OriginalTestData = MakeSameKPArguments(test2path)
OriginalDevnData = MakeSameKPArguments(dev2path)

MakeThreshold = []
for d in OriginalTrainData:
  KP = d[0]
  KP_embeddings = embedder.encode([KP])[0]
  Arguments = d[1]
  ArgumentsEmbeddings = []
  for a in Arguments:
    AEmbed = embedder.encode([a])[0]
    ArgumentsEmbeddings.append(AEmbed)
  MakeThreshold.append([KP_embeddings, ArgumentsEmbeddings])
for d in OriginalDevnData:
  KP = d[0]
  KP_embeddings = embedder.encode([KP])[0]
  Arguments = d[1]
  ArgumentsEmbeddings = []
  for a in Arguments:
    AEmbed = embedder.encode([a])[0]
    ArgumentsEmbeddings.append(AEmbed)
  MakeThreshold.append([KP_embeddings, ArgumentsEmbeddings])

MakeTest = []
for d in OriginalTestData:
  KP = d[0]
  KP_embeddings = embedder.encode([KP])[0]
  Arguments = d[1]
  ArgumentsEmbeddings = []
  for a in Arguments:
    AEmbed = embedder.encode([a])[0]
    ArgumentsEmbeddings.append(AEmbed)
  MakeTest.append([KP_embeddings, ArgumentsEmbeddings])

## EmbedRank with SBERT

If you want to use MMR, please change embedrank to embedranpp.
Then, it is better to change hyper-parameter to adapt your task.

In [17]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from IPython.display import HTML
from sklearn import manifold
from sklearn.metrics.pairwise import cosine_distances

%matplotlib inline


def ncossim(embs_1, embs_2, axis=0):
    sims = np.inner(embs_1, embs_2)
    std = np.std(sims, axis=axis)
    ex = np.mean((sims-np.min(sims, axis=axis))/np.max(sims, axis=axis), axis=axis)
    return 0.5 + (sims-ex)/std


def mmr(doc_emb, cand_embs, key_embs):
    param = 0.5
    scores = param * ncossim(cand_embs, doc_emb, axis=0)
    if key_embs is not None:
        scores -= (1-param) * np.max(ncossim(cand_embs, key_embs), axis=1).reshape(scores.shape[0], -1)
    return scores


def embedrank(doc_emb, sent_embs, n_keys, threshold):
    assert 0 < n_keys, 'Please `key_size` value set more than 0'
    assert n_keys < len(sent_embs), 'Please `key_size` value set lower than `#sentences`'
    sims = np.inner(doc_emb, sent_embs).reshape(-1)
    return np.argsort(-sims)[:n_keys]

"""
def embedrankpp(doc_emb, sent_embs, n_keys):
    assert 0 < n_keys, 'Please `key_size` value set more than 0'
    assert n_keys < len(sent_embs), 'Please `key_size` value set lower than `#sentences`'
    cand_idx = list(range(len(sent_embs)))
    key_idx = []
    while len(key_idx) < n_keys:
        cand_embs = sent_embs[cand_idx]
        key_embs = sent_embs[key_idx] if len(key_idx) > 0 else None
        scores = mmr(doc_emb, cand_embs, key_embs)
        key_idx.append(cand_idx[np.argmax(scores)])
        cand_idx.pop(np.argmax(scores))
    return key_idx
"""

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


"\ndef embedrankpp(doc_emb, sent_embs, n_keys):\n    assert 0 < n_keys, 'Please `key_size` value set more than 0'\n    assert n_keys < len(sent_embs), 'Please `key_size` value set lower than `#sentences`'\n    cand_idx = list(range(len(sent_embs)))\n    key_idx = []\n    while len(key_idx) < n_keys:\n        cand_embs = sent_embs[cand_idx]\n        key_embs = sent_embs[key_idx] if len(key_idx) > 0 else None\n        scores = mmr(doc_emb, cand_embs, key_embs)\n        key_idx.append(cand_idx[np.argmax(scores)])\n        cand_idx.pop(np.argmax(scores))\n    return key_idx\n"

In [53]:
#MakeTest: [KP_embeddings, ArgumentsEmbeddings]
#MakeThreshold: [KP_embeddings, ArgumentsEmbeddings]
RESULTS = []
key_size = 1
threshold = 0
for i in range(len(MakeTest)):
  TestList = MakeTest[i]
  KPEmbed = TestList[0]
  ArgEmbed = TestList[1]
  keys = embedrank(KPEmbed, ArgEmbed, key_size, threshold)
  KPOriginal = OriginalTestData[i][0]
  ArgumentsOriginal = OriginalTestData[i][1]
  RESULTS.append([KPOriginal, ArgumentsOriginal[keys[0]]])

# Score Metrics

### ROUGE SCORE

In [54]:
!pip install rouge

In [55]:
from rouge import Rouge 
rouge = Rouge()

In [56]:
#scores = rouge.get_scores(RESULTS[0][0], RESULTS[0][1])

In [32]:
#scores

[{'rouge-1': {'f': 0.5217391254442345, 'p': 0.5454545454545454, 'r': 0.5},
  'rouge-2': {'f': 0.1904761854875285, 'p': 0.2, 'r': 0.18181818181818182},
  'rouge-l': {'f': 0.45454544954545456,
   'p': 0.45454545454545453,
   'r': 0.45454545454545453}}]

In [57]:
RougeScores = []
for each_result in RESULTS:
  KP = each_result[0]
  Arg = each_result[1]
  scores = rouge.get_scores(KP, Arg)
  RougeScores.append([scores[0], KP, Arg])

In [58]:
R1_F1 = 0
R2_F1 = 0
RL_F1 = 0
R1_P = 0
R2_P = 0
RL_P = 0
R1_R = 0
R2_R = 0
RL_R = 0

MinR1 = 1.0
MinRL = 1.0
MaxR1 = 0.0
MaxRL = 0.0

for r in RougeScores:
  RougeDirectory = r[0]
  KP = r[1]
  Arg = r[2]

  R1 = RougeDirectory['rouge-1']
  R2 = RougeDirectory['rouge-2']
  RL = RougeDirectory['rouge-l']

  R1_F1 = R1_F1 + R1['f']
  R2_F1 = R2_F1 + R2['f']
  RL_F1 = RL_F1 + RL['f']

  R1_P = R1_P + R1['p']
  R2_P = R2_P + R2['p']
  RL_P = RL_P + RL['p']
  
  R1_R = R1_R + R1['r']
  R2_R = R2_R + R2['r']
  RL_R = RL_R + RL['r']

  if MinR1 > R1_F1:
    MinR1 = R1_F1
    KPMinR1 = KP
    ArgMinR1 = Arg
  if MinRL > RL_F1:
    MinRL = RL_F1
    KPMinRL = KP
    ArgMinRL = Arg
  if MaxR1 < R1_F1:
    MaxR1 = R1_F1
    KPMaxR1 = KP
    ArgMaxR1 = Arg
  if MaxRL < RL_F1:
    MaxRL = RL_F1
    KPMaxRL = KP
    ArgMaxRL = Arg

LengthOfScores = len(RougeScores)

In [59]:
print("===Rouge-1===")
print("F1       : " + str(R1_F1/LengthOfScores))
print("Precision: " + str(R1_P/LengthOfScores))
print("Recall   : " + str(R1_R/LengthOfScores))
print("===Rouge-2===")
print("F1       : " + str(R2_F1/LengthOfScores))
print("Precision: " + str(R2_P/LengthOfScores))
print("Recall   : " + str(R2_R/LengthOfScores))
print("===Rouge-L===")
print("F1       : " + str(RL_F1/LengthOfScores))
print("Precision: " + str(RL_P/LengthOfScores))
print("Recall   : " + str(RL_R/LengthOfScores))

===Rouge-1===
F1       : 0.17943774452068129
Precision: 0.27442161663473136
Recall   : 0.1448008254583646
===Rouge-2===
F1       : 0.049684630011168185
Precision: 0.07899504866717981
Recall   : 0.03960188141302141
===Rouge-L===
F1       : 0.18038845327110314
Precision: 0.26704042769616537
Recall   : 0.14676986008799853


In [60]:
print("Argument which yeilds Minimum F on Rouge-1")
print("Minimum F on R1: " + str(MinR1))
print("KP: " + KPMinR1)
print("Arg: " + ArgMinR1)
print("")

print("Argument which yeilds Minimum F on Rouge-L")
print("Minimum F on RL: " + str(MinRL))
print("KP: " + KPMinRL)
print("Arg: " + ArgMinRL)
print("")

print("Argument which yeilds Max F on Rouge-1")
print("Max F on R1: " + str(MaxRL))
print("KP: " + KPMaxRL)
print("Arg: " + ArgMaxRL)
print("")

print("Argument which yeilds Max F on Rouge-L")
print("Max F on RL: " + str(MaxRL))
print("KP: " + KPMaxRL)
print("Arg: " + ArgMaxRL)

Argument which yeilds Minimum F on Rouge-1
Minimum F on R1: 0.5217391254442345
KP: Parents should be permitted to choose the education of their children
Arg: parents should have the right to choose how to educate their children

Argument which yeilds Minimum F on Rouge-L
Minimum F on RL: 0.45454544954545456
KP: Parents should be permitted to choose the education of their children
Arg: parents should have the right to choose how to educate their children

Argument which yeilds Max F on Rouge-1
Max F on R1: 11.003695649537292
KP: Restrictions on migration would benefit people in the rural areas economically/socially
Arg: urbanization leaves rural communities impoverished. it should be stemmed so that rural areas continue to thrive and develop.

Argument which yeilds Max F on Rouge-L
Max F on RL: 11.003695649537292
KP: Restrictions on migration would benefit people in the rural areas economically/socially
Arg: urbanization leaves rural communities impoverished. it should be stemmed so tha

===Rouge-1===
* F1       : 0.1794
* Precision: 0.2744
* Recall   : 0.1448

===Rouge-2===
* F1       : 0.0497
* Precision: 0.0790
* Recall   : 0.0396

===Rouge-L===
* F1       : 0.1804
* Precision: 0.2670
* Recall   : 0.1468

"""
===Rouge-1===
F1       : 0.17943774452068129
Precision: 0.27442161663473136
Recall   : 0.1448008254583646
===Rouge-2===
F1       : 0.049684630011168185
Precision: 0.07899504866717981
Recall   : 0.03960188141302141
===Rouge-L===
F1       : 0.18038845327110314
Precision: 0.26704042769616537
Recall   : 0.14676986008799853
"""

### BLUE SCORE

In [61]:
!pip install bleu

  Created wheel for bleu: filename=bleu-0.3-cp36-none-any.whl size=5801 sha256=c91d2e36c035adfe807c7749acfa6bf7b0eadd9c7b9836e62e17bf28607ac643
  Stored in directory: /root/.cache/pip/wheels/a9/95/e7/cb43a1c509c38fedbee6223963e34a51a94d8991f3b3e1888e
  Created wheel for efficiency: filename=efficiency-0.4-cp36-none-any.whl size=19617 sha256=0951821cf30f628665b214dd1edfffa04631fde40ca0b92add8f4c46b5201dd5
  Stored in directory: /root/.cache/pip/wheels/1c/67/60/11693f94754d75dabfadb120c7ba380e7ef2aed8d3dccd00bf
Successfully built bleu efficiency


In [62]:
from bleu import list_bleu

KPs = [r[1] for r in RougeScores]
Args = [r[2] for r in RougeScores]

BlueScore = list_bleu([Args], KPs)

In [63]:
BlueScore #

0.98

Blue Score Result:
0.98 (below 1.0%!!!)

###BERTScore

In [64]:
!pip install bert-score

     |████████████████████████████████| 61kB 3.7MB/s 


In [70]:
from bert_score import score
def calc_bert_score(cands, refs):
    Precision, Recall, F1 = score(cands, refs, lang="en", verbose=True)
    return Precision.numpy().tolist(), Recall.numpy().tolist(), F1.numpy().tolist()

In [75]:
KPs = [r[1] for r in RougeScores]
Args = [r[2] for r in RougeScores]
P, R, F1 = calc_bert_score(Args, KPs)

calculating scores...
computing bert embedding.



computing greedy matching.



done in 1.44 seconds, 42.36 sentences/sec


In [76]:
print(sum(P)/len(P))
print(sum(R)/len(R))
print(sum(F1)/len(F1))

0.8866757232634748
0.8992083258316165
0.8926650442060877


In [77]:
minPrecision = 1.0
maxPrecision = 0.0
minRecall = 1.0
maxRecall = 0.0
minF1 = 1.0
maxF1 = 0.0
for idx in range(len(P)):
  if minPrecision > P[idx]:
    minPrecision = P[idx]
    minPrecisionKP = KPs[idx]
    minPrecisionArg = Args[idx]
  if maxPrecision < P[idx]:
    maxPrecision = P[idx]
    maxPrecisionKP = KPs[idx]
    maxPrecisionArg = Args[idx]
  if minRecall > R[idx]:
    minRecall = R[idx]
    minRecallKP = KPs[idx]
    minRecallArg = Args[idx]
  if maxRecall < R[idx]:
    maxRecall = R[idx]
    maxRecallKP = KPs[idx]
    maxRecallArg = Args[idx]
  if minF1 > F1[idx]:
    minF1 = F1[idx]
    minF1KP = KPs[idx]
    minF1Arg = Args[idx]
  if maxF1 < F1[idx]:
    maxF1 = F1[idx]
    maxF1KP = KPs[idx]
    maxF1Arg = Args[idx]

In [81]:
print("=== minPrecision ===")
print(minPrecision)
print(minPrecisionKP)
print(minPrecisionArg)

print("=== maxPrecision ===")
print(maxPrecision)
print(maxPrecisionKP)
print(maxPrecisionArg)

print("=== minRecall ===")
print(minRecall)
print(minRecallKP)
print(minRecallArg)

print("=== maxRecall ===")
print(maxRecall)
print(maxRecallKP)
print(maxRecallArg)

print("=== minF1===")
print(minF1)
print(minF1KP)
print(minF1Arg)

print("=== maxF1===")
print(maxF1)
print(maxF1KP)
print(maxF1Arg)

=== minPrecision ===
0.8237141370773315
Marriage benefits children
marriage between two people creates a committed union forming the basis of a strong and secure union that is a safe setting to nurture children. it is a statement to the world of a bond between two peoples.
=== maxPrecision ===
0.9610263109207153
Parents should be permitted to choose the education of their children
parents should have the right to choose how to educate their children
=== minRecall ===
0.8283309936523438
Affirmative action is ineffective
it does not really work so is a waste of resources
=== maxRecall ===
0.9792323708534241
Urbanization harms the environment
urbanization is causing more pollution and hurting the environment
=== minF1===
0.8405146598815918
Austerity extend recessions
the costs of an austerity regime is too high and would destroy the economy. unemployment rates would surge as spending decreases leading to economic destruction.
=== maxF1===
0.9648058414459229
Parents should be permitted to 


BERT Score
*   Precision: 0.8866757232634748
*   Recall       : 0.8992083258316165
*   F-1: 0.8926650442060877


### Show Text Embedding on 2D

In [ ]:
display_sents = []
print(KPOriginal)
print(ArgumentsOriginal[keys[0]])
print("===========")
for i, s in enumerate(ArgumentsOriginal):
    line = '<font color="#CD5C5C"><strong>' + s + '</strong></font>' if i in keys else s
    display_sents.append(line)
HTML('              '.join(display_sents))

In [ ]:
# plot
mds = manifold.MDS(n_components=2, dissimilarity="precomputed")
embs = np.concatenate((doc_emb, sent_embs), 0)
dist_matrix = cosine_distances(embs, embs)
pns = mds.fit_transform(dist_matrix)
fixed_pns = pns - pns[0]

keys_idx = [idx + 1 for idx in keys]
other_idx = [idx for idx in range(1, len(ArgumentsOriginal)+1) if idx not in keys_idx]
plt.scatter(fixed_pns[0,0], fixed_pns[0,1], color='green',  marker='*', s=150, label='document')
plt.scatter(fixed_pns[keys_idx,0], fixed_pns[keys_idx, 1], color='blue', label='key sentences')
plt.scatter(fixed_pns[other_idx,0], fixed_pns[other_idx, 1], color='white', edgecolors='black', label='other sentences')
plt.xlim(-0.7, 0.7)
plt.ylim(-0.7, 0.7)

for i,(x,y) in enumerate(fixed_pns[1:], 1):
    plt.annotate(str(i), (x+0.02,y+0.02))
    
plt.legend()
plt.show()

# LexRank